In [1]:
import pandas as pd

In [2]:
import pybedtools
pybedtools.set_bedtools_path('/tscc/nfs/home/ekofman/miniconda3/envs/marine_environment/bin/')

In [3]:
b = pybedtools.BedTool()

# Integration tests

## Todo: 
* Make a test for single-end approach, using this bam: /tscc/projects/ps-yeolab3/ekofman/sailor2/data/Hugo-A1Aligned.sortedByCoord.out.md.subset.bam
    * All of the reverse reads map to positive strand genes
    * All of the forward reads map to negative strand genes

### Run tests

In [14]:
%%bash

pwd
echo "Running"

cd ..
pwd

tests_folder="tests/strandedness_tests/"

export MARINE=/tscc/projects/ps-yeolab3/ekofman/sailor2/
export mypython=/tscc/nfs/home/ekofman/miniconda3/envs/marine_environment/bin/python

for t in "F1R2_pair_test-single_end_mode_sailor" "F1R2_pair_test-single_end_mode" "unstranded_pair_test" "F1R2_pair_test" "F2R1_end_second_in_pair_test" "same_pos_dif_reads_test" "tax1bp3_chr17_3665556_read_test" "pair_test"
do
    echo $t
    echo "Removing old files..."
    rm $tests_folder$t/* -r

    echo "Running tests..."
    bash tests/strandedness_tests/scripts/$t.sh $mypython
   
done



/tscc/projects/ps-yeolab3/ekofman/sailor2/tests
Running
/tscc/projects/ps-yeolab3/ekofman/sailor2
F1R2_pair_test-single_end_mode_sailor
Removing old files...
Running tests...
Python is /tscc/nfs/home/ekofman/miniconda3/envs/marine_environment/bin/python
Assuming 16 cores available for multiprocessing. Set this to the number of available cores for optimal execution.
Arguments:
	BAM filepath:	/tscc/projects/ps-yeolab3/ekofman/sailor2//tests/strandedness_tests/bams/F1R2_pair.bam
	Annotation bedfile filepath:	/tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/hg38_gencode.v35.annotation.genes.bed
	Output folder:	/tscc/projects/ps-yeolab3/ekofman/sailor2//tests/strandedness_tests/F1R2_pair_test-single_end_mode_sailor
	Barcode whitelist:	None
	Strandedness:	2
	Barcode Tag:	None
	Paired End:	False
	Coverage only:	False
	Filtering only:	False
	Annotation only:	False
	Sailor outputs:	['CT', 'AG']
	Bedgraphs:	[]
	Minimum base quality:	0
	Minimum read quality:	0
	Minimum distance from end:	0


100%|██████████| 16/16 [00:05<00:00,  2.79it/s]


~~~~~~
!!!!SINGLE END!!!!!
~~~~~~~`
~~~~~~
!!!!SINGLE END!!!!!
~~~~~~~`
~~~~~~
!!!!SINGLE END!!!!!
~~~~~~~`


100%|██████████| 16/16 [00:05<00:00,  2.92it/s]


Concatenating results...
Done concatenating.
original    4
filtered    3
dtype: int64
Total time to calculate coverage: 0.094 minutes
Filtering..
	Number of edits after filtering:
	3
	Number of unique edit sites:
	3
Writing sites...

Adding strand-specific conversion...

3 sites being converted to SAILOR format...
Generating SAILOR-style bed outputs for conversion CT...
0 rows had coverage of 0 or more edits than coverage... filtering these out, but look into them...
1 final deduplicated C>T SAILOR-formatted sites
Generating SAILOR-style bed outputs for conversion AG...
0 rows had coverage of 0 or more edits than coverage... filtering these out, but look into them...
0 final deduplicated A>G SAILOR-formatted sites
Annotating sites with GTF information from /tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/hg38_gencode.v35.annotation.genes.bed...
Current memory usage 21.288251MB; Peak: 22.400467MB
Time elapsed: 12.35s
-------------------------------
Deleting intermediate files...
-

...

Concatenating results...
Done concatenating.
original    32
filtered    24
dtype: int64
Total time to calculate coverage: 0.101 minutes
Filtering..
	Number of edits after filtering:
	24
	Number of unique edit sites:
	10
Writing sites...

Adding strand-specific conversion...

Annotating sites with GTF information from /tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/hg38_gencode.v35.annotation.genes.bed...
Current memory usage 21.395534MB; Peak: 22.504722MB
Time elapsed: 12.82s
-------------------------------
Deleting intermediate files...
-------------------------------
++++++
Done!
++++++
tax1bp3_chr17_3665556_read_test
Removing old files...
Running tests...
Assuming 16 cores available for multiprocessing. Set this to the number of available cores for optimal execution.
Arguments:
	BAM filepath:	/tscc/projects/ps-yeolab3/ekofman/sailor2//tests/strandedness_tests/bams/tax1bp3_chr17_3665556_read.bam
	Annotation bedfile filepath:	/tscc/projects/ps-yeolab3/ekofman/sailor2//annotatio

100%|██████████| 16/16 [00:05<00:00,  2.97it/s]


Concatenating results...
Done concatenating.
original    6
filtered    6
dtype: int64
Total time to calculate coverage: 0.093 minutes
Filtering..
	Number of edits after filtering:
	6
	Number of unique edit sites:
	6
Writing sites...

Adding strand-specific conversion...

6 sites being converted to SAILOR format...
Generating SAILOR-style bed outputs for conversion CT...
0 rows had coverage of 0 or more edits than coverage... filtering these out, but look into them...
1 final deduplicated C>T SAILOR-formatted sites
Annotating sites with GTF information from /tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/hg38_gencode.v35.annotation.genes.bed...
Current memory usage 21.404582MB; Peak: 22.514748MB
Time elapsed: 12.41s
-------------------------------
Deleting intermediate files...
-------------------------------
++++++
Done!
++++++
pair_test
Removing old files...
Running tests...
Assuming 1 cores available for multiprocessing. Set this to the number of available cores for optimal ex

100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


Concatenating results...
Done concatenating.
original    6
filtered    4
dtype: int64
Total time to calculate coverage: 0.019 minutes
Filtering..
	Number of edits after filtering:
	4
	Number of unique edit sites:
	2
Writing sites...

Adding strand-specific conversion...

2 sites being converted to SAILOR format...
Generating SAILOR-style bed outputs for conversion CT...
0 rows had coverage of 0 or more edits than coverage... filtering these out, but look into them...
2 final deduplicated C>T SAILOR-formatted sites
Annotating sites with GTF information from /tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/cellranger-GRCh38-3.0.0.annotation.genes.bed...
Current memory usage 21.316828MB; Peak: 22.426777MB
Time elapsed: 3.06s
++++++
Done!
++++++


In [15]:
%%bash

pwd
echo "Running"

cd ..
pwd

export MARINE=/tscc/projects/ps-yeolab3/ekofman/sailor2/
export mypython=/tscc/nfs/home/ekofman/miniconda3/envs/marine_environment/bin/python

tests_folder="tests/singlecell_tests/"
for t in "only_5_cells_test" "long_read_sc_test"

do
    echo $t
    echo "Removing old files..."
    rm $tests_folder$t/* -r

    echo "Running old tests..."
    bash tests/singlecell_tests/scripts/$t.sh $mypython
   
done


/tscc/projects/ps-yeolab3/ekofman/sailor2/tests
Running
/tscc/projects/ps-yeolab3/ekofman/sailor2
only_5_cells_test
Removing old files...
Running old tests...
Assuming 16 cores available for multiprocessing. Set this to the number of available cores for optimal execution.
Arguments:
	BAM filepath:	/tscc/projects/ps-yeolab3/ekofman/sailor2//tests/singlecell_tests/bams/9_3000526_only_5_cells.bam
	Annotation bedfile filepath:	/tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/cellranger-mm10-3.0.0.annotation.genes.bed
	Output folder:	/tscc/projects/ps-yeolab3/ekofman/sailor2//tests/singlecell_tests/only_5_cells_test
	Barcode whitelist:	None
	Strandedness:	2
	Barcode Tag:	CB
	Paired End:	False
	Coverage only:	False
	Filtering only:	False
	Annotation only:	False
	Sailor outputs:	[]
	Bedgraphs:	[]
	Minimum base quality:	0
	Minimum read quality:	0
	Minimum distance from end:	0
	Maximum edits per read:	None
	Contigs:	9
	Number of intervals:	16
	Cores:	16
	Verbose:	False
	Keep intermediate 

100%|██████████| 16/16 [00:05<00:00,  2.77it/s]


	4 suffixes
Total time to concat and write bams: 0.097 minutes
Deleting overall_label_to_list_of_contents...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Calculating coverage at edited sites, minimum read quality is 0...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
	split 9_000_0_7787195, 0/16...
	split 9_010_77871950_85659145, 10/16...
Done grouping! Concatenating ...
Done concatenating!


100%|██████████| 4/4 [00:05<00:00,  1.44s/it]


Concatenating results...
Done concatenating.
original    99
filtered    99
dtype: int64
Total time to calculate coverage: 0.099 minutes
Filtering..
	Number of edits after filtering:
	99
	Number of unique edit sites:
	73
Writing sites...

Adding strand-specific conversion...

Annotating sites with GTF information from /tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/cellranger-mm10-3.0.0.annotation.genes.bed...
Current memory usage 21.628384MB; Peak: 22.735117MB
Time elapsed: 18.71s
-------------------------------
Deleting intermediate files...
-------------------------------
++++++
Done!
++++++
long_read_sc_test
Removing old files...
Running old tests...
Assuming 16 cores available for multiprocessing. Set this to the number of available cores for optimal execution.
Arguments:
	BAM filepath:	/tscc/projects/ps-yeolab3/ekofman/sailor2//examples/data/LR_single_cell.md.subset.filtered.sorted.bam
	Annotation bedfile filepath:	/tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/cell

100%|██████████| 4/4 [00:05<00:00,  1.26s/it]


	400 suffixes
Total time to concat and write bams: 0.247 minutes
Deleting overall_label_to_list_of_contents...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Calculating coverage at edited sites, minimum read quality is 0...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
	split 6_000_0_37434137, 0/4...
Done grouping! Concatenating ...
Done concatenating!


100%|██████████| 400/400 [00:08<00:00, 48.91it/s] 


Concatenating results...
Done concatenating.
original    3
filtered    3
dtype: int64
Total time to calculate coverage: 0.141 minutes
Filtering..
	Number of edits after filtering:
	3
	Number of unique edit sites:
	3
Writing sites...

Adding strand-specific conversion...

Annotating sites with GTF information from /tscc/projects/ps-yeolab3/ekofman/sailor2//annotations/cellranger-mm10-3.0.0.annotation.genes.bed...
Current memory usage 21.519682MB; Peak: 22.63154MB
Time elapsed: 29.99s
-------------------------------
Deleting intermediate files...
-------------------------------
++++++
Done!
++++++


# Integration test automatic checks

In [8]:
test_name_to_expectations = {
    "unstranded_pair_test": {
        "folder": "strandedness_tests",
        "expectations": [{
            "contig": "Citrine.dna",
            "position": 435,
            "count": 22,
            "coverage": 22,
            "num_rows": 1,
            "strand_conversion": "C>T",
            "strand": "+"
        }]
    },
    "pair_test": {
        "folder": "strandedness_tests",
        "expectations": [{
            "contig": "18",
            "position": 49491556,
            "count": 2,
            "coverage": 2,
            "num_rows": 1,
            "strand_conversion": "C>T",
            "strand": "-",
            "feature_name": "RPL17-C18orf32,RPL17",
            "feature_strand": "-,-"
        },
        {
            "contig": "18",
            "position": 49567494,
            "count": 2,
            "coverage": 2,
            "num_rows": 1,
            "strand_conversion": "C>T",
            "strand": "+",
            "feature_name": "LIPG",
            "feature_strand": "+"
        }]
    },
    
    "F1R2_pair_test": {
        "folder": "strandedness_tests",
        "expectations": [{
            "contig": "chr17",
            "position": 43044352,
            "count": 1,
            "coverage": 1,
            "conversion": "G>A",
            "num_rows": 1,
            "conversion": "G>A",
            "strand_conversion": "C>T",
            "strand": "-",
            "feature_name": "BRCA1",
            "feature_strand": "-"
        }]
    },
    
    "F1R2_pair_test-single_end_mode_sailor": {
        "folder": "strandedness_tests",
        "expectations": [{
            "contig": "chr17",
            "position": 43044352,
            "count": 1,
            "coverage": 2,
            "conversion": "G>A",
            "num_rows": 1,
            "conversion": "G>A",
            "strand_conversion": "C>T",
            "strand": "-",
            "feature_name": "BRCA1",
            "feature_strand": "-"
        }]
    },

    "F1R2_pair_test-single_end_mode": {
        "folder": "strandedness_tests",
        "expectations": [{
            "contig": "chr17",
            "position": 43044352,
            "count": 1,
            "coverage": 2,
            "conversion": "G>A",
            "num_rows": 1,
            "conversion": "G>A",
            "strand_conversion": "C>T",
            "strand": "-",
            "feature_name": "BRCA1",
            "feature_strand": "-"
        }]
    },

     "F2R1_end_second_in_pair_test": {
        "folder": "strandedness_tests",
        "expectations": [{
            "contig": "chr17",
            "position": 43001716,
            "count": 1,
            "coverage": 1,
            "conversion": "G>A",
            "strand_conversion": "G>A",
            "strand": "+",
            "feature_name": "RPL27"
        }]
    },
    "same_pos_dif_reads_test": {
        "folder": "strandedness_tests",
        "expectations": [{
            "contig": "chr17",
            "position": 83199872,
            "count": 9,
            "coverage": 9,
            "conversion": "C>G",
            "strand_conversion": "C>G",
            "strand": "+",
            "feature_name": "AC139099.2"
        }]
    },
    "tax1bp3_chr17_3665556_read_test": {
        "folder": "strandedness_tests",
        "expectations": [{
            "contig": "chr17",
            "position": 3665556,
            "num_rows": 1,
            "count": 1,
            "coverage": 1,
            "conversion": "G>A",
            "strand_conversion": "G>A",
            "strand": "+",
            #"feature_name": "AC139099.2"
        }]
    },

        "only_5_cells_test": {
        "folder": "singlecell_tests",
        "expectations": [{
            "contig": "9",
            "barcode": 	"GGGACCTTCGAGCCAC-1",
            "position": 3000524,
            "num_rows": 1,
            "count": 1,
            "coverage": 12,
            "conversion": "C>A",
            "strand_conversion": "G>T",
            "strand": "-"
        },
        {
            "contig": "9",
            "barcode": 	"GGGACCTTCGAGCCAC-1",
            "position": 3000525,
            "num_rows": 1,
            "count": 1,
            "coverage": 12,
            "conversion": "C>T",
            "strand_conversion": "G>A",
            "strand": "-"
        },
        {
            "contig": "9",
            "barcode": 	"GATCCCTCAGTAACGG-1",
            "position": 3000525,
            "num_rows": 1,
            "count": 1,
            "coverage": 4,
            "conversion": "C>G",
            "strand_conversion": "G>C",
            "strand": "-"
        }]
    },
    "long_read_sc_test": {
        "folder": "singlecell_tests",
        "expectations": [{
            "contig": "6",
            "barcode": 	"ENSMUST00000203816-AACGTGTTGGAGAGGG-16-G",
            "position": 115807969,
            "num_rows": 1,
            "count": 1,
            "coverage": 1,
            "conversion": "A>C",
            "strand_conversion": "T>G",
            "strand": "-",
            "feature_name": "Rpl32"
        },
        {
            "contig": "6",
            "barcode": 	"ENSMUST00000081840-AAGTCGTACCAGGCTC-40-C",
            "position": 115805653,
            "num_rows": 1,
            "count": 1,
            "coverage": 1,
            "conversion": "G>A",
            "strand_conversion": "C>T",
            "strand": "-",
            "feature_name": "Rpl32"
        },
        {
            "contig": "6",
            "barcode": 	"ENSMUST00000081840-AACGTGTTGGAGAGGG-40-G",
            "position": 115807015,
            "num_rows": 1,
            "count": 1,
            "coverage": 8,
            "conversion": "C>T",
            "strand_conversion": "G>A",
            "strand": "-",
            "feature_name": "Rpl32"
        }]
    }

}

failures = 0
for test_name, info in test_name_to_expectations.items():
    print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\nChecking results for {}".format(test_name))
    
    expectations_list = info.get("expectations")
    for expectations in expectations_list:
        print("\tExpecting: {}".format(expectations))
    
              
        folder = info.get("folder")
        
        contig = expectations.get("contig")
        barcode = expectations.get("barcode", None)
        
        position = expectations.get("position")
        
        final_filtered_site_info_annotated = "{}/{}/final_filtered_site_info_annotated.tsv".format(folder, test_name)
        final_filtered_site_info_annotated_df = pd.read_csv(final_filtered_site_info_annotated, sep='\t', index_col=0)
    
        row_of_interest = final_filtered_site_info_annotated_df[
            (final_filtered_site_info_annotated_df['position'] == position) &\
            (final_filtered_site_info_annotated_df['contig'].astype(str) == contig)
        ]
    
    
        if barcode:
            row_of_interest = row_of_interest[row_of_interest.barcode == barcode]
    
        failure = False
        try:
            assert(len(row_of_interest) == expectations.get("num_rows", 1))
        except Exception as e:
            print("Num rows expected: {}, was {}".format(expectations.get("num_rows", 1), len(row_of_interest)))
            failure = True
            
        for attribute in list(expectations.keys()):
            if attribute in ['count', 'coverage', 'conversion', 'strand', 'feature_name']:
                attribute_expectation = expectations.get(attribute)
                try:
                    assert(row_of_interest[attribute].iloc[0] == attribute_expectation)
                except Exception as e:
                    print("Exception: {} was {}".format(attribute, row_of_interest[attribute].iloc[0]))
                    failure = True
        if not failure:
            print("\n\t >>> {} passed! <<<\n".format(test_name))
        else:
            print("\n\t ~~~ {} FAILED! ~~~\n".format(test_name))
            failures += 1

print("There were {} failures".format(failures))


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Checking results for unstranded_pair_test
	Expecting: {'contig': 'Citrine.dna', 'position': 435, 'count': 22, 'coverage': 22, 'num_rows': 1, 'strand_conversion': 'C>T', 'strand': '+'}

	 >>> unstranded_pair_test passed! <<<


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Checking results for pair_test
	Expecting: {'contig': '18', 'position': 49491556, 'count': 2, 'coverage': 2, 'num_rows': 1, 'strand_conversion': 'C>T', 'strand': '-', 'feature_name': 'RPL17-C18orf32,RPL17', 'feature_strand': '-,-'}
Exception: count was 1

	 ~~~ pair_test FAILED! ~~~

	Expecting: {'contig': '18', 'position': 49567494, 'count': 2, 'coverage': 2, 'num_rows': 1, 'strand_conversion': 'C>T', 'strand': '+', 'feature_name': 'LIPG', 'feature_strand': '+'}

	 >>> pair_test passed! <<<


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Checking results for F1R2_pair_test
	Expecting: {'contig': 'chr17', 'position': 43044352, 'count': 1, '

IndexError: single positional indexer is out-of-bounds